In [2]:
population_size = 50
max_generations = 100
mutation_rate = 0.1
credit_hour_limit = 18
import random


In [3]:
class Course:
    def __init__(self, code, title, credit_hours, prerequisites=None):
        self.code = code
        self.title = title
        self.credit_hours = int(credit_hours)
        self.prerequisites = set() if prerequisites is None else set(prerequisites)

In [4]:
curriculum = {
    1: {
        "sc1201": Course("sc1201", "Applied Physics", "3", ""),
        "sc1001": Course("sc1001", "Calculus & Analytic Geometry", "3", ""),
        "hu1002": Course("hu1002", "English Composition & Comprehension", "3", ""),
        "cs1501": Course("cs1501", "Introduction to Information and Communication Technologies", "2", ""),
        "cs1001": Course("cs1001", "Programming Fundamentals", "4", " ")
    },
    2: {
        "hu1003": Course("hu1003", "Communication & Presentation Skills", "3", ""),
        "cs1502": Course("cs1502", "Digital Logic and Design", "4", ""),
        "hu1101": Course("hu1101", "Islamic Studies", "2", ""),
        "sc1002": Course("sc1002", "Multivariate Calculus", "2", "sc1001"),
        "cs2301": Course("cs2301", "Discrete Structures", "2", ""),
        "cs1002": Course("cs1002", "Programming Techniques", "2", "cs1001")
    },
    3: {
        "cs2503": Course("cs2503", "Computer Organization & Assembly Language", "4", ""),
        "cs2003": Course("cs2003", "Data Structure and Algorithms", "4", "cs1001"),
        "sc2003": Course("sc2003", "Differential Equations", "3", "sc1001"),
        "hu1102": Course("hu1102", "Pakistan Studies", "2", ""),
        "cs2004": Course("cs2004", "Object Oriented Programming", "4", "cs1001")
    },
    4: {
        "cs2201": Course("cs2201", "Introduction to Database Systems", "4", "cs2003"),
        "cs2504": Course("cs2504", "Operating Systems", "4", "cs2003"),
        "sc2004": Course("sc2004", "Probability and Statistics", "3", ""),
        "cs2101": Course("cs2101", "Software Engineering", "3", "")
    },
    5: {
        "cs4303": Course("cs4303", "Artificial Intelligence", "4", "cs2003"),
        "cs3005": Course("cs3005", "Design & Analysis of Algorithms", "3", "cs2003"),
        "cs3202": Course("cs3202", "Web Engineering", "4", "cs2003"),
        "cs3002": Course("cs3002", "Computer Networks", "3", "cs2003"),
        "cs3003": Course("cs3003", "Software Project Management", "3", "cs2101")
    },
    6: {
        "cs3701": Course("cs3701", "Computer Graphics", "4", "cs2004"),
        "cs4102": Course("cs4102", "Software Project-I", "4", "cs2004"),
        "cs3702": Course("cs3702", "Theory of Automata", "3", "cs2004"),
        "cs4001": Course("cs4001", "Digital Image Processing", "3", "cs2004"),
        "cs4602": Course("cs4602", "Data Warehousing & Data Mining", "3", "cs2201")
    },
    7: {
        "cs4701": Course("cs4701", "Introduction to Machine Learning", "3", "CS3005"),
        "cs4802": Course("cs4802", "Software Project-II", "3", "cs4102"),
        "cs4201": Course("cs4201", "Advanced Database Systems", "3", "cs2201"),
        "cs4302": Course("cs4302", "Software Quality Assurance", "3", "cs3003"),
        "cs4503": Course("cs4503", "Compiler Construction", "3", "cs3702"),
        "cs4004": Course("cs4004", "Parallel Computing", "3", "cs3002")
    },
    8: {
        "cs4801": Course("cs4801", "Final Year Project", "4", ""),
        "cs4902": Course("cs4902", "Software Project-III", "4", "cs4802"),
        "cs4502": Course("cs4502", "Advanced Operating Systems", "4", "cs2504"),
        "cs4603": Course("cs4603", "Human Computer Interaction", "3", ""),
        "cs4005": Course("cs4005", "Wireless Networks", "3", "cs3002")
    }
}

In [5]:
def crossover(parent1, parent2):
    # Select a random semester for crossover
    crossover_point = random.randint(1, min(len(parent1), len(parent2)))

    # Perform crossover
    offspring1 = {semester: parent1[semester] if semester < crossover_point else parent2[semester] for semester in parent1}
    offspring2 = {semester: parent2[semester] if semester < crossover_point else parent1[semester] for semester in parent2}

    return offspring1, offspring2


In [6]:
def mutate(chromosome):
    mutated_chromosome = chromosome.copy()

    # Select a random semester for mutation
    semester = random.choice(list(chromosome.keys()))

    # Select a random course in the semester for mutation
    courses = chromosome[semester]
    if courses:
        course_index = random.randint(0, len(courses) - 1)
        course = courses[course_index]

        # Replace the course with a random course from the curriculum
        new_course = random.choice(list(curriculum[semester].values()))
        mutated_chromosome[semester][course_index] = new_course

    return mutated_chromosome


In [7]:
def generate_chromosome(current_semester, failed_courses):
    chromosome = {}
    total_credit_hours = 0

    for semester in range(1, current_semester + 1):
        chromosome[semester] = []

        # Add courses from the curriculum to the chromosome
        for course_code, course in curriculum[semester].items():
            if course_code not in failed_courses:
                # Check if prerequisites are satisfied
                if course.prerequisites.issubset(chromosome[semester]):
                    # Check if credit hour limit is not exceeded
                    if total_credit_hours + int(course.credit_hours) <= credit_hour_limit:
                        chromosome[semester].append(course_code)
                        total_credit_hours += int(course.credit_hours)

    return chromosome


In [8]:
def calculate_fitness(chromosome, failed_courses):
    fitness = 0

    for semester, courses in chromosome.items():
        for course in courses:
            if course in failed_courses:
                fitness += 1

    return fitness


In [9]:
def genetic_algorithm(current_semester, failed_courses):
    population = []

    # Generate initial population
    for _ in range(population_size):
        chromosome = generate_chromosome(current_semester, failed_courses)
        population.append(chromosome)

    for generation in range(max_generations):
        # Evaluate fitness of each chromosome
        fitness_scores = [calculate_fitness(chromosome, failed_courses) for chromosome in population]

        # Select parents for next generation
        parents = random.choices(population, weights=[1 / (fitness + 1) for fitness in fitness_scores], k=2)

        # Perform crossover
        offspring1, offspring2 = crossover(parents[0], parents[1])

        # Perform mutation on each offspring
        offspring1 = mutate(offspring1)
        offspring2 = mutate(offspring2)

        # Replace old population with new generation
        population = [offspring1, offspring2] + random.choices(population[2:], k=population_size - 2)

    # Select the best chromosome from the final population
    best_chromosome = min(population, key=lambda chromosome: calculate_fitness(chromosome, failed_courses))
    return best_chromosome


In [11]:
current_semester = 3
failed_courses = ["cs2003", "cs2503", "sc1002"]

best_chromosome = genetic_algorithm(current_semester, failed_courses)
course_codes = [course.code for semester_courses in best_chromosome.values() for course in semester_courses]
print(course_codes)



['hu1002', 'cs1001', 'sc1001', 'sc1201', 'sc1002', 'hu1101']
